In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [12]:
df=pd.read_csv("municipality_bus_utilization.csv")

In [13]:
df.head()

,timestamp,municipality_id,usage,total_capacity
0,2017-06-04 07:59:42,9,454,1332
1,2017-06-04 07:59:42,8,556,2947
2,2017-06-04 07:59:42,4,1090,3893
3,2017-06-04 07:59:42,0,204,2813
4,2017-06-04 07:59:42,7,718,2019


In [14]:
df.tail()

,timestamp,municipality_id,usage,total_capacity
13065,2017-08-19 16:30:35,2,548,697
13066,2017-08-19 16:30:35,8,1193,2947
13067,2017-08-19 16:30:35,7,1354,2019
13068,2017-08-19 16:30:35,6,1680,3113
13069,2017-08-19 16:30:35,3,1189,1930


In [15]:
df.shape

(13070, 4)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13070 entries, 0 to 13069
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   timestamp        13070 non-null  object
 1   municipality_id  13070 non-null  int64 
 2   usage            13070 non-null  int64 
 3   total_capacity   13070 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 408.6+ KB


In [17]:
df.isnull().sum()

timestamp          0
municipality_id    0
usage              0
total_capacity     0
dtype: int64

In [18]:
df.describe()

,municipality_id,usage,total_capacity
count,13070.000000,13070.000000,13070.000000
mean,4.500000,876.002984,1972.800000
std,2.872391,679.287109,1144.074782
min,0.000000,11.000000,397.000000
25%,2.000000,403.000000,697.000000
50%,4.500000,692.000000,1974.500000
75%,7.000000,1190.000000,2947.000000
max,9.000000,3508.000000,3893.000000


In [19]:
df.corr()

,municipality_id,usage,total_capacity
municipality_id,1.000000,0.183785,0.173069
usage,0.183785,1.000000,0.682632
total_capacity,0.173069,0.682632,1.000000


In [20]:
df["timestamp"]=pd.to_datetime(df["timestamp"])  

In [21]:
df["year"]=2023-df["timestamp"].dt.year  
df["month"]=df["timestamp"].dt.month
df["day"]=df["timestamp"].dt.day 

In [22]:
df.head()

,timestamp,municipality_id,usage,total_capacity,year,month,day
0,2017-06-04 07:59:42,9,454,1332,6,6,4
1,2017-06-04 07:59:42,8,556,2947,6,6,4
2,2017-06-04 07:59:42,4,1090,3893,6,6,4
3,2017-06-04 07:59:42,0,204,2813,6,6,4
4,2017-06-04 07:59:42,7,718,2019,6,6,4


In [23]:
x=df[["municipality_id","total_capacity","year","month","day"]]
y=df["usage"] 

In [24]:
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [25]:
def regression(x,y):
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
    
    #Lineer Olanlar
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Lasso,Ridge
    from sklearn.linear_model import ElasticNet
    
    #Lineer olmayanlar
    from sklearn.tree import ExtraTreeRegressor
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.neighbors import KNeighborsRegressor
    
    x=pd.get_dummies(x,drop_first=True)
    
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
    
    Li=LinearRegression()
    La=Lasso()
    Ri=Ridge()
    EL=ElasticNet()
    ETR=ExtraTreeRegressor()
    GBR=GradientBoostingRegressor()
    KNR=KNeighborsRegressor()
    
    model_array=[Li,La,Ri,EL,ETR,GBR,KNR]
    model_names=["LinearRegression","Lasso","Ridge","ElasticNet","ExtraTreeRegressor","GradientBoostingRegressor","KNeighborsRegressor"]
    
    
    mean_squared_array=[]
    r2_score_array=[]
    mean_absolute_array=[]
    
    result=pd.DataFrame(columns=["mean_squared_error","r2_score","mean_absolute_error"],index=model_names)
    
    for i in model_array:
        i.fit(x_train,y_train)
        pred_i=i.predict(x_test)
        mean_squared_array.append((mean_squared_error(y_test,pred_i)**0.5))
        r2_score_array.append(r2_score(y_test,pred_i))
        mean_absolute_array.append(mean_absolute_error(y_test,pred_i))
        
    result.mean_squared_error=mean_squared_array
    result.r2_score=r2_score_array
    result.mean_absolute_error=mean_absolute_array
    
    return result.sort_values("r2_score",ascending=False)

In [26]:
regression(x,y)

,mean_squared_error,r2_score,mean_absolute_error
ExtraTreeRegressor,250.195271,0.863930,171.865794
KNeighborsRegressor,261.646653,0.851190,175.947743
GradientBoostingRegressor,326.590312,0.768149,217.039626
LinearRegression,496.746880,0.463620,359.564078
Ridge,496.746983,0.463619,359.564090
Lasso,496.770800,0.463568,359.552948
ElasticNet,497.482194,0.462030,359.853250
